In [ ]:
from pyomo.environ import *
import numpy as np

# Parámetros
T = 24  # Horizonte de planificación (horas)
E_max = 100  # Capacidad máxima de la batería (MWh)
P_max = 50   # Potencia máxima de carga/descarga (MW)
DoD = 0.8    # Profundidad de descarga permitida (80%)
eta_c = 0.9  # Eficiencia de carga
eta_d = 0.9  # Eficiencia de descarga
E_0 = 50     # Estado inicial de carga de la batería (MWh)
E_final = 50 # Estado final de carga de la batería (MWh)
precios = np.random.uniform(20, 100, T)  # Precios aleatorios para cada hora ($/MWh)

# Modelo Pyomo
model = ConcreteModel()

# Variables de decisión
model.P_carga = Var(range(T), domain=NonNegativeReals, bounds=(0, P_max))
model.P_descarga = Var(range(T), domain=NonNegativeReals, bounds=(0, P_max))
model.E = Var(range(T), domain=NonNegativeReals, bounds=(E_max * (1 - DoD), E_max))

# Función objetivo: Maximizar el beneficio por arbitraje de precios
model.obj = Objective(expr=sum(precios[t] * model.P_descarga[t] - precios[t] * model.P_carga[t] for t in range(T)), sense=maximize)

# Restricción de balance de energía
model.balance = ConstraintList()
for t in range(T):
    if t == 0:
        model.balance.add(model.E[t] == E_0 + eta_c * model.P_carga[t] - model.P_descarga[t] / eta_d)
    else:
        model.balance.add(model.E[t] == model.E[t-1] + eta_c * model.P_carga[t] - model.P_descarga[t] / eta_d)

# Restricción de no carga y descarga simultánea
model.no_simultaneo = ConstraintList()
for t in range(T):
    model.no_simultaneo.add(model.P_carga[t] * model.P_descarga[t] == 0)

# Restricción de estado final de la batería
model.final_energy = Constraint(expr=model.E[T-1] == E_final)

# Resolver el modelo con CBC (o puedes cambiar a otro solver como Gurobi o CPLEX)
solver = SolverFactory('cbc')
solver.solve(model, tee=True)

# Imprimir resultados
print("Beneficio máximo:", model.obj())
for t in range(T):
    print(f"Hora {t}: Carga = {model.P_carga[t]()} MW, Descarga = {model.P_descarga[t]()} MW, Energía almacenada = {model.E[t]()} MWh")
